In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

BASE_URL = "https://www2.hm.com"
headers = {"User-Agent": "Mozilla/5.0"}

def get_all_product_links(listing_url, max_pages=3):
    links = []
    for page in range(1, max_pages+1):
        url = f"{listing_url}?page={page}"
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")

        product_anchors = soup.select("a.product-item-link")
        for a in product_anchors:
            product_url = BASE_URL + a['href']
            if product_url not in links:
                links.append(product_url)

        time.sleep(random.uniform(1, 2))
    return links

def scrape_hm_product(url):
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")

        name = soup.select_one("h1.product-item-headline").get_text(strip=True)
        price = soup.select_one("span.price-value").get_text(strip=True).replace("$", "")
        description = soup.select_one("p.pdp-description-text")
        description = description.get_text(strip=True) if description else "N/A"
        image = soup.select_one("img.product-detail-main-image")["src"]
        image_url = "https:" + image if image.startswith("//") else image

        return {
            "image": image_url,
            "brand": "H&M",
            "name": name,
            "description": description,
            "price": float(price)
        }

    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

# Example for Women section
listing_url = "https://www2.hm.com/en_us/women/products/view-all.html"
product_links = get_all_product_links(listing_url, max_pages=3)  # Adjust max_pages as needed

# Scrape each product
hm_data = []
for link in product_links:
    result = scrape_hm_product(link)
    if result:
        hm_data.append(result)
    time.sleep(random.uniform(1, 2))

# Save results
df = pd.DataFrame(hm_data)
df.to_csv("hm_women_products.csv", index=False)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))